## Simple MLP for Prediction of Ozone Layer Concentration using Scisat Data

This notebook contains a basic implementation of a neural network that aims to predict average ozone concentrations given a geographic location (lat/lon) and the day of year. This is accomplished using the keras python package. If you are unfamiliar with Scisat data, it is recommended that you begin by working through the initial tutorial notebook found in this repository.

Input Parameters: Lat/Lon of measurement, Day of year (1-365) of measurement
Output (target): Average ozone concentration of measurement across all altitudes

In [2]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px

First, let's download the ozone concentration dataset from the CSA's open data portal. The resulting pandas dataframe will be processed and split into training and testing datasets.

In [3]:
df = pd.read_csv('https://donnees-data.asc-csa.gc.ca/users/OpenData_DonneesOuvertes/pub/SCISAT/Data%20format%20CSV%202004-2020/ACEFTS_L2_v4p1_O3.csv', engine= 'python')

Next, we will perform some cleaning and preparation on the dataset. If you have completed the initial tutorial, this is essentially the same process as the cleaning in that notebook.

In [4]:
# Remove scientifically infeasible values for concentration measurement columns
df[df.iloc[:,df.columns.get_loc('0.5'):df.columns.get_loc('149.5')+1]>1e-5]=np.nan
df[df.iloc[:,df.columns.get_loc('0.5'):df.columns.get_loc('149.5')+1]<0]=np.nan

# Remove values which are too far from the mean value for each column (Gaussian distribution assumed)
std=df.std(skipna=True, numeric_only=True)
mn=df.mean(skipna=True, numeric_only=True)
maxV = mn+3*std
minV = mn-3*std
df[df.gt(maxV) | df.lt(minV)] = np.nan

# Recalculate mean on altitude for each measurement after removing aberrant values
df['Alt_Mean'] = df.iloc[:,df.columns.get_loc('0.5'):df.columns.get_loc('149.5')+1].T.mean(skipna=True)

# Take note of the max concentration value for use in normalization of model training data
true_max = df['Alt_Mean'].max()

# Create a column with the sample dates converted to day of year (doy)
df['date'] = pd.to_datetime(df['date'])
df['doy'] = df['date'].dt.dayofyear

# Simplify the dataframe to only the columns required and drop any rows with NaN values
df = df[['doy','lat','long','Alt_Mean']]
df = df.dropna()

The data is now normalized in order to optimize the performance of the model. After, we randomly sample 80% of the data to be used in training, and assign the remaining 20% of samples for the testing and validation step.

In [5]:
# Scale columns to range (0,1)
for col in df.columns:
    df[col] = MinMaxScaler().fit_transform(df[col].to_numpy().reshape(-1,1))

# Split dataset into train and test sets
df_train = df.sample(frac=0.8,random_state=0)
df_test = df.drop(df_train.index)

# Separate input columns from target column
df_x_train = df_train[['doy','lat','long']]
df_x_test = df_test[['doy','lat','long']]
df_y_train = df_train['Alt_Mean']
df_y_test = df_test['Alt_Mean']

We are now ready to build our neural network model. Using keras, we will create a model with two dense hidden layers and an output layer with a linear activation function.

In [6]:
model_1 = keras.Sequential()
model_1.add(layers.Dense(32, input_shape=(3,)))
model_1.add(layers.Dense(64, activation='sigmoid'))
model_1.add(layers.Dense(32, activation='sigmoid'))
model_1.add(layers.Dense(1, activation='linear'))

With our newly defined neural net, we can now comile and fit the model to the data.

In [7]:
model_1.compile(optimizer='adam', loss='mse')
model_1.fit(df_x_train,df_y_train, batch_size=32, epochs=20)

Epoch 1/20
2461/2461 [==============================] - 15s 5ms/step - loss: 0.0157
Epoch 2/20
2461/2461 [==============================] - 13s 5ms/step - loss: 0.0036
Epoch 3/20
2461/2461 [==============================] - 11s 5ms/step - loss: 0.0028
Epoch 4/20
2461/2461 [==============================] - 11s 5ms/step - loss: 0.0027
Epoch 5/20
2461/2461 [==============================] - 12s 5ms/step - loss: 0.0024
Epoch 6/20
2461/2461 [==============================] - 12s 5ms/step - loss: 0.0022
Epoch 7/20
2461/2461 [==============================] - 11s 5ms/step - loss: 0.0022
Epoch 8/20
2461/2461 [==============================] - 11s 4ms/step - loss: 0.0022
Epoch 9/20
2461/2461 [==============================] - 11s 4ms/step - loss: 0.0020
Epoch 10/20
2461/2461 [==============================] - 11s 4ms/step - loss: 0.0019
Epoch 11/20
2461/2461 [==============================] - 11s 4ms/step - loss: 0.0018
Epoch 12/20
2461/2461 [==============================] - 10s 4ms/step - lo

In [8]:
# Use the trained model to predict ozone concentrations from the test dataset
results = model_1.predict(df_x_test)
# Scale the results back to the original range
results = results*true_max

616/616 [==============================] - 2s 3ms/step


To visualize the results of the predictions, we will prepare the data for several plots.

In [9]:
# Sort the columns by day of year
df_x_test_temp = df_x_test.reset_index()
doy_sorted_idx = list(df_x_test_temp['doy'].sort_values().index)
doy_sorted = df_x_test_temp['doy'][doy_sorted_idx].apply(lambda x: int(x*365))
results_sortedby_doy = results[doy_sorted_idx]
y_test_sortedby_doy = (df_y_test*true_max).reset_index()['Alt_Mean'][doy_sorted_idx]
lat_sortedby_doy = ((df_x_test['lat']*180)-90).reset_index()['lat']
lon_sortedby_doy = ((df_x_test['long']*360)-180).reset_index()['long']

In [10]:
# Create dataframe for plots
df_fig = pd.DataFrame()
df_fig['doy'] = doy_sorted
df_fig['results'] = results_sortedby_doy.flatten()
df_fig['target'] = y_test_sortedby_doy
df_fig['lat'] = lat_sortedby_doy
df_fig['long'] = lon_sortedby_doy


The first plot will show the test results compared to the actual ozone concentration values, aggregated by day of year.

In [11]:
# Aggregate dataframe by the day of year, taking the mean
df_tmp = df_fig.groupby(['doy']).agg('mean')
# Create and display the plot
fig = px.line(df_tmp, y=['results','target'], labels={'value':'Ozone concentration [ppv]','doy':'Day of year'}, title='Model Results by Day of Year')
fig.show() 

The second plot will separate the result and target values between the northern and southern hemispheres. This will allow us to visualize how the model is able to predict accurately not only by the time of year, but also given the latitude and longitude of the sample. It also provides an interesting visualization of the annual changes in ozone concentration for each hemisphere respectively.

In [12]:
df_fig['target_north'] = [row['target'] if row['lat'] > 0 else np.nan for (idx, row) in df_fig.iterrows()]
df_fig['target_south'] = [row['target'] if row['lat'] <= 0 else np.nan for (idx, row) in df_fig.iterrows()]
df_fig['results_north'] = [row['results'] if row['lat'] > 0 else np.nan for (idx, row) in df_fig.iterrows()]
df_fig['results_south'] = [row['results'] if row['lat'] <= 0 else np.nan for (idx, row) in df_fig.iterrows()]
df_tmp = df_fig.groupby(['doy']).agg('mean')
fig2 = px.line(df_tmp, y=['results_north','results_south','target_north','target_south'], labels={'value':'Ozone concentration [ppv]','doy':'Day of year'}, title='Model Results by Day of Year Separated by Hemisphere')
fig2.show() 

### Next steps

If you would like to practice working with this dataset and MLP model, here are a few ideas of additions to the project:

- Calculate metrics such as mean squared error and the R2 value of the model predictions
  - These values can be compared to simple benchmark models to determine the validity of our model. An example would be comparing with a naive model that predicts the mean value every time.
- Create baseline models for comparison
  - When creating and evaluating a regression model, it can be helpful to create quick, simple models to create a baseline to compare the performance. Often, you may even find that simple models perform just as well or even better than more complex neural networks! Some good models to try include random forests and SVM's, both of which can be implemented quickly using python libraries such as scikit-learn.
- Create a different model that uses this dataset
  - Think of another machine learning application that can be explored with Scisat data. This example completely ignores altitude, perhaps that can be incorporated?